# Practice - Lecture 24

During this lecture we are going to use the library "benchmarking_qrc" to learn about quantum reservoir computing. But, we will also use numpy to allow new learners to experiment and built the own code.

In [28]:
# Only for local use
import os
import sys

repo_path = os.path.dirname(os.getcwd())
package_path = repo_path + "/src"
sys.path.append(package_path)

In [29]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pathlib
from scipy.linalg import lstsq

import benchmarking_qrc as qrc
from benchmarking_qrc import run_memory_capacity as mc

### Exercise 0 - Define your first Qreservoir

To define the initial state you can use `qrc.reservoir.initial_state(num_sites, dim_site)` or `numpy`.

In [33]:
def initial_reservoir(num_sites, dim_site):
    """Density matrix with all states at |0>

    Args:
        num_sites (int): Number of sites in the system
        dim_site (int): Hilbert dimension of each site.
    """
    # Put your code here: Prepare the state |00><00|
    return qrc.reservoir.initial_state(num_sites, dim_site)

In [34]:
def check_solution(solution, exp_solution):
    
    if solution.shape == exp_solution.shape:
        if np.all(solution == exp_solution):
            print("Correct!")
        else:
            print("Try again!")
    else:
        print("Try again!")

In [36]:
#solution = initial_reservoir(2, 2)
solution = initial_reservoir(2, 2)
exp_solution = np.array([[1, 0, 0, 0], [0]*4, [0]*4, [0]*4])

check_solution(solution, exp_solution)

Correct!


Now, we will define the dynamics of the system following this quadratic hamiltonian 
$$ H = \sum_{i,j=1}^{N}J_{ij}a_{i}^{\dagger}a_{j} $$
and the evolution operator associated to it: $e^{-iH\Delta t}$.

You can define the $J_{ij}$ coefficients with: 
- `qrc.hamiltonian.get_coefficients(num_sites, coef_range=[0, 1], seed=1)`.  

Define a quadratic hamiltonian with:  
- `qrc.hamiltonian.quadratic_op(num_sites, is_bosonic=..., dimensions=..., coefficients=...)`

and build the evolution operator with:
-  `qrc.hamiltonian.get_evolution_op(hamiltonian, dt=...)`


In [38]:
def my_hamiltonian(num_sites, dim_site):
    """Define previous quadratic hamiltonian

    Args:
        num_sites (_type_): Number of sites in the system.
        dim_site (_type_): Hilbert dimension of each site.
    """
    # Put your code here: Define J_ij coefficients
    J_ij = qrc.hamiltonian.get_coefficients(num_sites, coef_range=[0, 1], seed=1) 
    
    # Put your code here: Define quadratic hamiltonian
    return qrc.hamiltonian.quadratic_op(num_sites, is_bosonic=False, dimensions=dim_site, coefficients=J_ij)


def evolution_op(hamiltonian, delta_t):
    """Define the evolution operator related to the previous hamiltonian
    and let it evolves for 10 units.

    Args:
        hamiltonian (np.array): Matrix describing system dynamics.
        delta_t (float): Time we let the system evolve.
    """
    # Put your code here: Define evolution operator
    return qrc.hamiltonian.get_evolution_op(hamiltonian, dt=delta_t)
    
    

In [46]:
solution_hamiltonian = my_hamiltonian(2, 2)
solution_evolution = evolution_op(my_hamiltonian(2, 2), 10)


exp_sol_hamiltonian = np.array([
    [0.+0.j]*4, 
    [0.+0.j, 0.72+0.j, 0.+0.j  , 0.+0.j  ], 
    [0.+0.j, 0.+0.j  , 0.42+0.j, 0.+0.j  ], 
    [0.+0.j, 0.+0.j  , 0.+0.j  , 1.14+0.j]
])
exp_sol_evolution = np.array([
   [ 1.+0.j, 0.  +0.j ,  0.  +0.j  ,  0.  +0.j  ], 
   [ 0.+0.j, 0.61-0.8j,  0.  -0.j  ,  0.  +0.j  ],
   [ 0.+0.j, 0.  -0.j , -0.52+0.86j,  0.  +0.j  ],
   [ 0.+0.j, 0.  +0.j ,  0.  +0.j  ,  0.37+0.93j]
])

print("Checking solution hamiltonian...")
check_solution(np.around(solution_hamiltonian.todense(), 2), exp_sol_hamiltonian)
print("Checking evolution operator...")
check_solution(np.around(solution_evolution, 2), exp_sol_evolution)


Checking solution hamiltonian...
Correct!
Checking evolution operator...
Correct!


### Exercise 1 - Echo State Property

### Exercise 2 - Evaluate model

You will train a quantum system to recall past inputs and evaluate the performance with the memory capacity.

In [ ]:
# 1. Posar un exemple amb una box blava: quins son els inputs i quin són els output

# 2. Donarlis ja train_inputs, train_targets, test_X i test_y


### Exercise 3 - Define your own model

Build your own reservoir and evaluate performance.

Examples:
 - Ising model
 - Bose-Hubbard
 - Previous hamiltonian with qubits.

Have fun and explore!

_Recall: If you feel stuck, you can check: hamiltonian.py_